In [2]:
'''THIS SCRIPT RUNS ACCESSIBILITY ANALYSES (MINIMUM-DISTANCE AND COVERAGE METHODS)'''

#service area distance in metres
service_area = 500

'''CREATE FILE PATHS'''

#You cannot use relative file paths in ArcGIS, therefore base_folder must be manually set. Base folder = overall project folder
BASE_FOLDER = r""

bin_folder = BASE_FOLDER+r"\bin"

bin_gdb = bin_folder + "\dissertation_gdb.gdb"

data_folder = BASE_FOLDER+ r"\input_data"

outputs = BASE_FOLDER + r"\outputs"

xml_file = BASE_FOLDER+r"\input_data\XML\london_network_template.xml"

'''CREATE SHAPEFILE PATHS'''

#The overall network dataset. This file path needs to be "''"
network_dataset = r"%s\london_road_network" %bin_gdb

#The actual network
network = r"%s\london_road_network\london_road_network_dataset" %bin_gdb

#Network junctions
network_junctions = r"%s\london_road_network\london_road_network_dataset_Junctions" %bin_gdb

#this chargers layer will be 'snapped' to the road network, rather than being geographically seperate
chargers_snap = r"%s\NCR_chargers_snap" %outputs

chargers = r"%s\NCR_chargers.shp" %outputs

#The edges of the road network as .shp. This one has to be in between ''
road_split_apostrophe= r"'%s\osm_roads_split.shp'" %bin_folder

#The junctions of the road network as .shp
dissolved_points=r"%s\road_points_dissolved.shp" %bin_folder

lsoa_centres = r"%s\lsoa_cropped_centres.shp" %outputs

lsoa_cropped = r"%s\lsoa_stats_cropped.shp" %outputs

#Output- shortest routes
service_routes_dissolved = r"%s\service_routes_dissolved.shp" %bin_folder

buffer = r"%s\buffer.shp" %bin_folder

#Output. Can only be written to a gdb (geodatabase)
buffer_summarise = r"%s\buffer_summarise" %bin_gdb

#snap chargers to road network
arcpy.edit.Snap(chargers_snap, road_split_apostrophe+" EDGE '500 Meters'")

#snap lsoa centres to road network
arcpy.edit.Snap(lsoa_centres, road_split_apostrophe+" EDGE '500 Meters'")

'''MINIMUM-DISTANCE ACCESSIBILITY'''

#make closest facility netork
arcpy.na.MakeClosestFacilityAnalysisLayer(network, "Closest Facility", "New Travel Mode", "TO_FACILITIES", None, 1, None, "LOCAL_TIME_AT_LOCATIONS", "START_TIME", "ALONG_NETWORK", None, "NO_DIRECTIONS", "SKIP")

#add facilities to network (EV chargers)
arcpy.na.AddLocations("Closest Facility", "Facilities", chargers_snap, "Name ChargeDevi #;CurbApproach # 0;Attr_Length # 0;Cutoff_Length # #", "5000 Meters", None, "osm_roads_split SHAPE;road_points_dissolved SHAPE;london_road_network_dataset_Junctions NONE", "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", None, None)

#add incidents to network (LSOA centres)
arcpy.na.AddLocations("Closest Facility", "Incidents", lsoa_centres, "Name LSOA11CD #;TargetFacilityCount # #;CurbApproach # 0;Attr_Length # 0;Cutoff_Length # #", "5000 Meters", None, "osm_roads_split SHAPE;road_points_dissolved SHAPE;london_road_network_dataset_Junctions NONE", "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", None, None)

#run analysis: shortest route between incident and closest facility
arcpy.na.Solve("Closest Facility", "SKIP", "TERMINATE", None, '')

#add new LSOA11CD field to the created routes file
arcpy.management.AddField(r"Closest Facility\Routes", "LSOA11CD", "TEXT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

#input this field with LSOA11CD values
arcpy.management.CalculateField(r"Closest Facility\Routes", "LSOA11CD", "!Name![0:9]", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

#then join this field to our main statistics layer, so that we have the total length of shortest route 
arcpy.management.JoinField(lsoa_cropped, "LSOA11CD", r"Closest Facility\Routes", "LSOA11CD", "Total_Length", "NOT_USE_FM", None)

'''COVERAGE-BASED ACCESSIBILITY'''
#make service area layer
arcpy.na.MakeServiceAreaAnalysisLayer(network, "Service Area", "New Travel Mode", "FROM_FACILITIES", [service_area], None, "LOCAL_TIME_AT_LOCATIONS", "LINES", "STANDARD", "OVERLAP", "RINGS", "500 Meters", None, None, "SKIP")

#add facilities to service area layer (LSOA Centres)
arcpy.na.AddLocations("Service Area", "Facilities", lsoa_centres, "Name LSOA11CD #;CurbApproach # 0;Attr_Length # 0;Attr_New_Cost # 0;Breaks_Length # #;Breaks_New_Cost # #", "5000 Meters", None, "osm_roads_split SHAPE;road_points_dissolved SHAPE;london_road_network_dataset_Junctions NONE", "MATCH_TO_CLOSEST", "CLEAR", "NO_SNAP", "5 Meters", "EXCLUDE", None, "ALLOW")

#run the analysis: create 500m paths around each facility
arcpy.na.Solve("Service Area", "SKIP", "TERMINATE", None, '')

#dissolve lines based on Facility ID as there are lots of seperate lines for each facility
arcpy.analysis.PairwiseDissolve(r"Service Area\Lines", service_routes_dissolved, "FacilityID", None, "MULTI_PART", '')

#join Facilities layer to these created service routes so that the LSOA codes are transferred to the rotues
arcpy.management.JoinField(service_routes_dissolved, "FacilityID", r"Service Area\Facilities", "ObjectID", "Name", "NOT_USE_FM", None)

#run pairwise buffer of 25m around each 500m path, 25m being the (rough) depth of housing from the road
arcpy.analysis.PairwiseBuffer(service_routes_dissolved, buffer, "25 Meters", "NONE", None, "PLANAR", "0 Meters")

#count chargers within this buffer zone
arcpy.analysis.SummarizeWithin("buffer", chargers, buffer_summarise, "KEEP_ALL", None, "ADD_SHAPE_SUM", "SQUAREKILOMETERS", None, "NO_MIN_MAJ", "NO_PERCENT", None)

#join this value (number of chargers) to our main statistics layer
arcpy.management.JoinField(lsoa_cropped, "LSOA11CD", buffer_summarise, "Name", "Point_Count", "NOT_USE_FM", None)




<Result 'C:\\Users\\Captain Haddock\\Documents\\dissertation_test\\outputs\\lsoa_stats_cropped.shp'>